[View in Colaboratory](https://colab.research.google.com/github/MarkDaoust/models/blob/autopgraph-guide/samples/core/guide/autograph_control_flow.ipynb)

##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoGraph: Easy control flow for graphs 

<table class="tfo-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/models/blob/master/samples/core/guide/autograph_control_flow.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /><span>Run in Google Colab</span></a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/models/blob/master/samples/core/guide/autograph_control_flow.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /><span>View source on GitHub</span></a></td></table>

[AutoGraph](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/autograph/README.md) helps you write complicated graph code using just plain Python -- behind the scenes, AutoGraph automatically transforms your code into the equivalent TF graph code. We support a large chunk of the Python language, which is growing. [Please see this document for what we currently support, and what we're working on](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/autograph/LIMITATIONS.md).

In [0]:
! pip install tf-nightly

from __future__ import division, print_function, absolute_import

import tensorflow as tf
from tensorflow.contrib import autograph

import matplotlib.pyplot as plt

Here's a quick example of how it works. Autograph can convert functions like this:

In [0]:
def g(x):
  if x > 0:
    x = x * x
  else:
    x = 0.0
  return x

Into graph-compatible functions like this:

In [0]:
print(autograph.to_code(g))

You can take code written for eager execution and run it in graph mode. You get the same results, but with all the benfits of graphs:

In [0]:
print('Original value: %2.2f' % g(9.0))  

Generate a graph-version and call it:

In [0]:
tf_g = autograph.to_graph(g)

with tf.Graph().as_default():  
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
  g_ops = tf_g(tf.constant(9.0))
  with tf.Session() as sess:
    print('Autograph value: %2.2f\n' % sess.run(g_ops))  

## Automatically converting control flow

AutoGraph can convert a large chunk of the Python language into equivalent graph-construction code, and we're adding new supported language features all the time. In this section, we'll give you a taste of some of the functionality in AutoGraph.
AutoGraph will automatically convert most Python control flow statements into their correct graph equivalent.  
  

We support common statements like `while`, `for`, `if`, `break`, `return` and more. You can even nest them as much as you like. Imagine trying to write the graph version of this code by hand:


In [0]:
# Continue in a loop
def f(l):
  s = 0
  for c in l:
    if c % 2 > 0:
      continue
    s += c
  return s

print('Original value: %d' % f([10,12,15,20]))

tf_f = autograph.to_graph(f)

with tf.Graph().as_default():  
  with tf.Session():
    print('Graph value: %d\n\n' % tf_f(tf.constant([10,12,15,20])).eval())

In [0]:
print(autograph.to_code(f))

Try replacing the `continue` in the above code with `break` -- AutoGraph supports that as well!  

## Decorator

If you don't need easy access to the original python function use the `convert` decorator:

In [0]:
@autograph.convert()
def fizzbuzz(num):
  if num % 3 == 0 and num % 5 == 0:
      print('FizzBuzz')
  elif num % 3 == 0:
      print('Fizz')
  elif num % 5 == 0:
      print('Buzz')
  else:
      print(num)
  return num

In [0]:
with tf.Graph().as_default():  
  # The result works like a regular op: takes tensors in, returns tensors.
  # You can inspect the graph using tf.get_default_graph().as_graph_def()
  input = tf.placeholder(tf.int32)
  result = fizzbuzz(input)
  with tf.Session() as sess:
    sess.run(result, feed_dict={input:10})    
    sess.run(result, feed_dict={input:11})    
    sess.run(result, feed_dict={input:12})    
    sess.run(result, feed_dict={input:13})    
    sess.run(result, feed_dict={input:14}) 
    sess.run(result, feed_dict={input:15}) 
    

### Assert

Let's try some other useful Python constructs, like `print` and `assert`. We automatically convert Python `assert` statements into the equivalent `tf.Assert` code.  

In [0]:
def f(x):
  assert x != 0, 'Do not pass zero!'
  return x * x

tf_f = autograph.to_graph(f)

with tf.Graph().as_default():  
  with tf.Session():
    try:
      print(tf_f(tf.constant(0)).eval())
    except tf.errors.InvalidArgumentError as e:
      print('Got error message:\n%s' % e.message)

### Print

You can also use plain Python `print` functions in in-graph

In [0]:
@autograph.convert()
def f(n):
  if n >= 0:
    while n < 5:
      n += 1
      print(n)
  return n
    
with tf.Graph().as_default():
  with tf.Session():
    f(tf.constant(0)).eval()

### Lists

Appending to lists in loops also works (we create a `TensorArray` for you behind the scenes)

In [0]:
def f(n):
  z = []
  # We ask you to tell us the element dtype of the list
  z = autograph.utils.set_element_type(z, tf.int32)
  for i in range(n):
    z.append(i)
  # when you're done with the list, stack it
  # (this is just like np.stack)
  return autograph.stack(z) 

tf_f = autograph.to_graph(f)

with tf.Graph().as_default():  
  with tf.Session():
    print(tf_f(tf.constant(3)).eval())

### Nested If statement

In [0]:
@autograph.convert()
def nearest_odd_square(x):
  if x > 0:
    x = x * x
    if x % 2 == 0:
      x = x + 1
  return x

with tf.Graph().as_default():  
  with tf.Session() as sess:
    print(sess.run(nearest_odd_square(tf.constant(4))))
    print(sess.run(nearest_odd_square(tf.constant(5))))
    print(sess.run(nearest_odd_square(tf.constant(6))))

### While loop

In [0]:
@autograph.convert()
def square_until_stop(x, y):
  while x < y:
    x = x * x
  return x
    
with tf.Graph().as_default():  
  with tf.Session() as sess:
    print(sess.run(square_until_stop(tf.constant(4), tf.constant(100))))

### Break from loop

In [0]:
@autograph.convert()
def argwhere_cumsum(x, threshold):
  current_sum = 0.0
  idx = 0
  for i in range(len(x)):
    idx = i
    if current_sum >= threshold:
      break
    current_sum += x[i]
  return idx

N = 10
with tf.Graph().as_default():  
  with tf.Session() as sess:
    idx = argwhere_cumsum(tf.ones(N), tf.constant(float(N/2)))
    print(sess.run(idx))

## Advanced example: A training, loop in-graph

Writing control flow in AutoGraph is easy, so running a training loop in a TensorFlow graph should be easy as well!  

Here, we show an example of training a simple Keras model on MNIST, where the entire training process -- loading batches, calculating gradients, updating parameters, calculating validation accuracy, and repeating until convergence -- is done in-graph.

### Download data

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

### Define the model

In [0]:
def mlp_model(input_shape):
  model = tf.keras.Sequential((
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(100, activation='relu', input_shape=input_shape),
      tf.keras.layers.Dense(100, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')))
  model.build()
  return model


def predict(m, x, y):
  y_p = m(x)
  losses = tf.keras.losses.categorical_crossentropy(y, y_p)
  l = tf.reduce_mean(losses)
  accuracies = tf.keras.metrics.categorical_accuracy(y, y_p)
  accuracy = tf.reduce_mean(accuracies)
  return l, accuracy


def fit(m, x, y, opt):
  l, accuracy = predict(m, x, y)
  opt.minimize(l)
  return l, accuracy


def setup_mnist_data(is_training, batch_size):
  if is_training:
    ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    ds = ds.shuffle(batch_size * 10)
  else:
    ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  ds = ds.repeat()
  ds = ds.batch(batch_size)
  return ds


def get_next_batch(ds):
  itr = ds.make_one_shot_iterator()
  image, label = itr.get_next()
  x = tf.to_float(image)/255.0
  y = tf.one_hot(tf.squeeze(label), 10)
  return x, y 

### Define the training loop

In [0]:
# TODO: this fails silently (training does not converge) if I put the `convert` decorator up here.
def train(train_ds, test_ds, hp):
  m = mlp_model((28 * 28,))
  opt = tf.train.MomentumOptimizer(hp.learning_rate, 0.9)
  
  # We'd like to save our losses to a list. In order for AutoGraph
  # to convert these lists into their graph equivalent,
  # we need to specify the element type of the lists.
  train_losses = []
  train_losses = autograph.utils.set_element_type(train_losses, tf.float32)
  test_losses = []
  test_losses = autograph.utils.set_element_type(test_losses, tf.float32)
  train_accuracies = []
  train_accuracies = autograph.utils.set_element_type(train_accuracies, tf.float32)
  test_accuracies = []
  test_accuracies = autograph.utils.set_element_type(test_accuracies, tf.float32)
  
  # This entire training loop will be run in-graph.
  i = tf.constant(0)
  while i < hp.max_steps:
    train_x, train_y = get_next_batch(train_ds)
    test_x, test_y = get_next_batch(test_ds)
    # add get next
    step_train_loss, step_train_accuracy = fit(m, train_x, train_y, opt)
    step_test_loss, step_test_accuracy = predict(m, test_x, test_y)
    if i % (hp.max_steps // 10) == 0:
      print('Step', i, 'train loss:', step_train_loss, 'test loss:',
            step_test_loss, 'train accuracy:', step_train_accuracy,
            'test accuracy:', step_test_accuracy)
    train_losses.append(step_train_loss)
    test_losses.append(step_test_loss)
    train_accuracies.append(step_train_accuracy)
    test_accuracies.append(step_test_accuracy)
    i += 1
  
  # We've recorded our loss values and accuracies 
  # to a list in a graph with AutoGraph's help.
  # In order to return the values as a Tensor, 
  # we need to stack them before returning them.
  return (autograph.stack(train_losses), autograph.stack(test_losses),  
          autograph.stack(train_accuracies), autograph.stack(test_accuracies))

Now build the graph and run the training loop:

In [0]:
with tf.Graph().as_default() as g:
  hp = tf.contrib.training.HParams(
      learning_rate=0.05,
      max_steps=500,
  )
  train_ds = setup_mnist_data(True, 50)
  test_ds = setup_mnist_data(False, 1000)
  tf_train = autograph.to_graph(train)
  (train_losses, test_losses, train_accuracies,
   test_accuracies) = tf_train(train_ds, test_ds, hp)

  init = tf.global_variables_initializer()
  
with tf.Session(graph=g) as sess:
  sess.run(init)
  (train_losses, test_losses, train_accuracies,
   test_accuracies) = sess.run([train_losses, test_losses, train_accuracies,
                                test_accuracies])
  
plt.title('MNIST train/test losses')
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.xlabel('Training step')
plt.ylabel('Loss')
plt.show()
plt.title('MNIST train/test accuracies')
plt.plot(train_accuracies, label='train accuracy')
plt.plot(test_accuracies, label='test accuracy')
plt.legend(loc='lower right')
plt.xlabel('Training step')
plt.ylabel('Accuracy')
plt.show()